In [1]:
import datetime
import Model
import numpy as np
import pandas as pd
import math
from multiprocessing import Pool
import os
import warnings
import time
import matplotlib.pyplot as plt
from importlib import reload

reload(Model)
warnings.simplefilter('ignore', FutureWarning)

# positive numbers, start should be lower than end
# a rebalance index represents the month before the rebalancing takes place
# so returns are calculated starting at rebalanceIndex + 1
def rebalanceIndexes(startIndex, endIndex):
    indexes = list(range(startIndex, endIndex, 3))
    return (indexes)


# get ln returns for an equally balanced portfolio of stocks
def getReturns(portfolio, index, length):
	returns = 0
	print(portfolio)
	print(-1 * index + 1)
	print(-1 * index + length)
	nancount = 0
	for stock in portfolio:
		indReturn = Model.rateOfReturn(Model.retrieveData(stock, 'Last Price', -1 * index + 1, -1 * index + length, []))
		print(stock + ": " + str(indReturn))
		if (not math.isnan(indReturn)):
			returns += indReturn
		else:
			nancount += 1
	if len(portfolio) - nancount == 0:
		total = 0
	else:
		total = returns/(len(portfolio)-nancount)
	print("Start Date: " + str(Model.convertIndexToDate(-1 * index + 1)))
	print("Total Return: " +str(total))
	return (total)


# make a portfolio with predicted probabilities higher than a hardcoded threshold
def makePortfolio(treeTuple):
	featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
	addedStocks, probabilities = Model.predict_probabilities(treeTuple[1], startIndex = -1 * treeTuple[0] - 11, endIndex = -1 * treeTuple[0], features = featureList, sector = "Materials")
	probabilityThreshold = 0.8
	stockTuples = zip(addedStocks, probabilities)
	stockTuples = list(filter(lambda x: x[1][1] > probabilityThreshold, stockTuples))
	if len(stockTuples) == 0:
		print("Start Date: " + str(Model.convertIndexToDate(-1 * treeTuple[0] + 1)))
		print("No portfolio, probabilities lower than threshold of " + str(probabilityThreshold))
		return 0
	stocks, probabilities = zip(*stockTuples)
	return(getReturns(stocks, treeTuple[0], 3))

In [2]:
indexes = []
for i in rebalanceIndexes(4,56):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

18
[array([-219, -216, -213, -210, -207, -204, -201, -198, -195, -192, -189,
       -186, -183, -180, -177, -174, -171, -168, -165, -162, -159, -156,
       -153, -150, -147, -144, -141, -138, -135, -132, -129, -126, -123,
       -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,  -90,
        -87,  -84,  -81,  -78,  -75,  -72,  -69,  -66,  -63,  -60,  -57,
        -54,  -51,  -48,  -45,  -42,  -39,  -36,  -33,  -30,  -27,  -24,
        -21]), array([-222, -219, -216, -213, -210, -207, -204, -201, -198, -195, -192,
       -189, -186, -183, -180, -177, -174, -171, -168, -165, -162, -159,
       -156, -153, -150, -147, -144, -141, -138, -135, -132, -129, -126,
       -123, -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,
        -90,  -87,  -84,  -81,  -78,  -75,  -72,  -69,  -66,  -63,  -60,
        -57,  -54,  -51,  -48,  -45,  -42,  -39,  -36,  -33,  -30,  -27,
        -24]), array([-225, -222, -219, -216, -213, -210, -207, -204, -201, -198, -195,
       -192, -189

In [3]:
print("# of multiprocess cpus: " + str(os.cpu_count()))
sector = "Materials"
featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
forestList = []
for ind in indexes:
	randForest = Model.buildWithIndexesTripleClass(modelType = Model.randomForestClassifier, indexes = ind, target= 'Rate of Return', features = featureList, featureLength = 12,\
									targetLength = 3, sector = sector, percentileTarget = 90, percentileAvoid = 10, verbose = True)
	forestList.append(randForest)
print(forestList)

# of multiprocess cpus: 8
Finished data retrieval, starting model training. Time taken: 44.62077283859253 seconds.
Finished fitting. Time taken: 9.776010990142822 seconds.
Finished data retrieval, starting model training. Time taken: 37.30610203742981 seconds.
Finished fitting. Time taken: 8.083638191223145 seconds.
Finished data retrieval, starting model training. Time taken: 37.36656880378723 seconds.
Finished fitting. Time taken: 9.395695924758911 seconds.
Finished data retrieval, starting model training. Time taken: 45.797460079193115 seconds.
Finished fitting. Time taken: 10.203490018844604 seconds.
Finished data retrieval, starting model training. Time taken: 44.563124895095825 seconds.
Finished fitting. Time taken: 8.518394947052002 seconds.
Finished data retrieval, starting model training. Time taken: 39.0790650844574 seconds.
Finished fitting. Time taken: 8.355069875717163 seconds.
Finished data retrieval, starting model training. Time taken: 38.1267352104187 seconds.
Finished

In [4]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsList = pool.map(makePortfolio, zip(rebalanceIndexes(4,56), forestList))
print(returnsList)

# of multiprocess cpus: 8
('NGVT',)
Total Return: -0.0926124774875614
-3
-1
NGVT: -0.0926124774875614
Start Date: 2017-11-30
('CC', 'CF', 'KOP', 'KRO', 'NGVT', 'SXC')
Total Return: 0.19214439918115475
-6
-4
CC: 0.1429377979467561
CF: 0.2701087624505756
KOP: 0.21391744794091716
KRO: 0.2287655585942927
NGVT: 0.12325565548961137
SXC: 0.17388117266477554
Start Date: 2017-08-31
('CC', 'TROX')
Total Return: 0.20867978947904509
-9
-7
CC: 0.1744134003477007
TROX: 0.24294617861038947
Start Date: 2017-05-31
('HNH',)
-12
-10
HNH: 0.19948936041632015
Start Date: 2017-02-28
Total Return: 0.19948936041632015
('CF', 'KS', 'TMST', 'TROX')
TMST: 0.06493401254078845
-15
-13
CF: 0.19837491648384065
KS: 0.16021574076983747
TROX: 0.0989910673723462
Start Date: 2016-11-30
Total Return: 0.1306289342917032
('ZEUS',)
Start Date: 2016-08-31
-18
-16
ZEUS: 0.17774133014616433
Total Return: 0.17774133014616433
('BCC', 'GORO', 'HAYN', 'KOP')
Total Return: 0.3162751992549733
-21
-19
BCC: 0.16835985198689052
GORO: 0.